In [1]:
import json
from pythreejs import *
from pythreejs import ObjectLoader
import ipywidgets
from IPython.display import display
import numpy as np

In [2]:
def getPositions(morphology, geometryIndex):
    return morphology["geometries"][geometryIndex]["data"]["attributes"]["position"]["array"]

def getPositionArray(positions):
    positionArray = []
    for i in range(0,len(positions),3):
        pos3d = [positions[i], positions[i+1], positions[i+2]]
        positionArray.append(pos3d)        
    return positionArray

def getMeanCoordinate(vertexArray):
    return np.mean(vertexArray, axis=0)

In [3]:
#with open("morphology2.json") as f:
with open("morphology.json") as f:
    morphology = json.load(f)

scale_factor = 0.1

positionArray = []
indexArray = []
vertexArray = []

currentIndex = 0
for i in range(0, len(morphology["geometries"])):        
    positions = getPositions(morphology, i)
    positionArray = getPositionArray(positions)
    vertexArray.append(positionArray[0])
    for j in range(1,len(positionArray)-1):
        vertexArray.append(positionArray[j])
        vertexArray.append(positionArray[j])
    vertexArray.append(positionArray[-1])

mean = getMeanCoordinate(vertexArray)
offset = -scale_factor * mean
    
vertices = BufferAttribute(
    array=scale_factor * np.array(vertexArray,dtype=np.float32),
    normalized=False)

geometry = BufferGeometry(
    attributes={'position': vertices},
)

geometry.exec_three_obj_method('translate',offset[0], offset[1], offset[2])

material = LineBasicMaterial()
material.linewidth = 1
material.color = "#ff0000"

edges = LineSegments(geometry, material)

In [4]:
view_width = 600
view_height = 600

In [5]:
sphere = Mesh(
    SphereBufferGeometry(1, 32, 16),
    MeshStandardMaterial(color='red')
)

In [6]:
cube = Mesh(
    BoxBufferGeometry(1, 1, 1),
    MeshPhysicalMaterial(color='green'),
    position=[2, 0, 4]
)

In [7]:
camera = PerspectiveCamera( position=[100, 0, 0], aspect=view_width/view_height)
camera.up = [0,0,1] 
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

In [8]:
children=[camera, key_light, ambient_light, edges]
scene = Scene(children=children, background="#FAEBD7")
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

In [9]:
renderer

UmVuZGVyZXIoY2FtZXJhPVBlcnNwZWN0aXZlQ2FtZXJhKHBvc2l0aW9uPSgxMDAuMCwgMC4wLCAwLjApLCBxdWF0ZXJuaW9uPSgwLjAsIDAuMCwgMC4wLCAxLjApLCBzY2FsZT0oMS4wLCAxLjDigKY=
